In [1]:
# Dependencies and Setup
import pandas as pd
import locale

# Set number and currency formatting
locale.setlocale(locale.LC_ALL,'en_US')
locale._override_localeconv = {"n_sign_posn": 3}

# File to Load (Remember to Change These)
data_csv = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(data_csv)
purchase_data_df = pd.DataFrame(purchase_data)

In [2]:
# Formatting Variables
currency = "${:.2f}".format
percentage = "{:.2f}%".format

In [3]:
# Group dataframe by "SN" and count number of unique names
player_unique_df = purchase_data_df.drop_duplicates(subset="SN")
player_count = len(player_unique_df)

# Display total player count as dataframe and format column name
player_count_df = pd.DataFrame([player_count])
player_count_df.columns = ["Total Players"]

player_count_df

,Total Players
0,576


In [4]:
# Calculations for Purchasing Analysis = # Unique Items, Average Purchase Price, # of Purchases, and Total Revenue
unique_items = len(purchase_data_df["Item ID"].unique())
average_price = purchase_data_df["Price"].mean()
total_purchase_count = len(purchase_data_df["Purchase ID"].unique())
total_revenue = purchase_data_df["Price"].sum()

unique_items, average_price, total_purchase_count,total_revenue

#Define variable for and print summary table
summary = {
    "Number of Unique Items":unique_items,
    "Average Purchase Price":locale.currency(average_price,grouping=True),
    "Number of Purchases":total_purchase_count,
    "Total Revenue":locale.currency(total_revenue,grouping=True)}

purchasing_analysis_df = pd.DataFrame([summary])

purchasing_analysis_df

,Number of Unique Items,Average Purchase Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [5]:
# Calculate player count by unique SN and gender
gender_count = player_unique_df.groupby("Gender").count()

gender_df = pd.DataFrame(gender_count)
gender_df = gender_df.rename(columns={"SN":"Total Count"})

# Change above player count to percentages
gender_percentage = (gender_df["Total Count"] / player_count)*100
gender_percentage = gender_percentage.apply("{:.2f}%".format)
gender_percentage_df = pd.DataFrame(gender_percentage)
gender_percentage_df = gender_percentage_df.rename(columns={"Total Count":"Percentage of Players"})

# Merge 2 dataframes to show count and percentage. Source: https://www.dataindependent.com/pandas/pandas-merge/

gender_demographics_df = pd.merge(gender_df["Total Count"],gender_percentage_df["Percentage of Players"],on="Gender")

gender_demographics_df

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [6]:
# Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
gender_sorted_df = purchase_data_df.groupby(["Gender"])
gender_purch_count = gender_sorted_df["Gender"].count()
gender_average = gender_sorted_df["Price"].mean()
total_purchase_value = gender_sorted_df["Price"].sum()
avg_total_per = total_purchase_value/(gender_df["Total Count"])

# Create a summary data frame to hold the results
gender_summary = {"Purchase Count":gender_purch_count,"Average Purchase Price":gender_average,"Total Purchase Value":total_purchase_value,"Avg Total Purchase per Person":avg_total_per}

# Edit columns for cleaner formatting
gender_summary_df = pd.DataFrame(gender_summary)
gender_summary_df["Purchase Count"] = gender_purch_count
gender_summary_df["Average Purchase Price"] = locale.currency(gender_average,grouping=True
gender_summary_df["Total Purchase Value"] = locale.currency(total_purchase_value,grouping=Tru)
gender_summary_df["Avg Total Purchase per Person"] = locale_currency(avg_total_per,grouping=True)

# Display the summary data frame
gender_summary_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [7]:
# Establish bins for ages, categorize players by age group
bins = [0,9,14,19,24,29,34,39,100]
bin_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

# Calculate player count and percentages by age group, include formatting
player_age = pd.cut(player_unique_df["Age"],bins,labels = bin_names,include_lowest=True)
age_count = player_age.value_counts()
age_percentage = (age_count/player_count)*100
age_population = {"Total Count":age_count,"Percentage of Players":age_percentage.apply("{:.2f}%".format)}

# Create and display a summary data frame to hold the results, with index sorted from youngest to oldest.
age_df = pd.DataFrame(age_population).sort_index(axis=0)
age_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [8]:
# Bin the purchase_data data frame by age
purchase_data_df["Age Ranges"] = pd.cut(purchase_data_df["Age"],bins,labels = bin_names)

ages_grouped = purchase_data_df.groupby(by="Age Ranges")

age_purch_count = ages_grouped["Purchase ID"].count()
age_average = ages_grouped["Price"].mean()
age_total_purchase = ages_grouped["Price"].sum()
age_avg_total_per = (age_total_purchase/age_count)

age_purch_summary = {"Purchase Count":age_purch_count,"Average Purchase Price":age_average,"Total Purchase Value":age_total_purchase,"Avg Total Purchase per Person":age_avg_total_per}

age_summary_df = pd.DataFrame(age_purch_summary, columns = ["Purchase Count","Average Purchase Price","Total Purchase Value","Avg Total Purchase per Person"])
age_summary_df["Purchase Count"] = age_purch_count
age_summary_df["Average Purchase Price"] = age_average.apply("${:.2f}".format)
age_summary_df["Total Purchase Value"] = age_total_purchase.apply("${:.2f}".format)
age_summary_df["Avg Total Purchase per Person"] = age_avg_total_per.apply("${:.2f}".format)

age_summary_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [9]:
# Index DataFrame
purchase_data_indexed = purchase_data_df.set_index(["SN"])

# Run basic calculations to obtain the results in the table below
topspenders_df = purchase_data_indexed.groupby("SN")
topspenders_purch_count = topspenders_df["Purchase ID"].count()
topspenders_purch_avg = topspenders_df["Price"].mean()
topspenders_total_purch = topspenders_df["Price"].sum()

# Create a summary data frame to hold the results
summary_df = pd.DataFrame({
    "Purchase Count":topspenders_purch_count,
    "Average Purchase Price":topspenders_purch_avg,
    "Total Purchase Value":topspenders_total_purch})

# Sort the total purchase value column in descending order
summary_sorted = summary_df.sort_values(by = "Total Purchase Value",ascending = False)

# Optional: give the displayed data cleaner formatting
summary_sorted["Average Purchase Price"] = topspenders_purch_avg.apply("${:.2f}".format)
summary_sorted["Total Purchase Value"] = topspenders_total_purch.apply("${:.2f}".format)

# Display a preview of the summary data frame
summary_sorted.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [10]:
# * Retrieve the Item ID, Item Name, and Item Price columns
items_purchase_data_indexed = purchase_data_df.set_index(["Item ID","Item Name"])
popularitems_df = items_purchase_data_indexed.groupby(["Item ID","Item Name"])

# * Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value
popularitems_purch_count = popularitems_df["Purchase ID"].count()
popularitems_purch_avg = popularitems_df["Price"].mean()
popularitems_total_purch = popularitems_df["Price"].sum()

# * Create a summary data frame to hold the results
items_summary = {"Purchase Count":popularitems_purch_count,
    "Average Purchase Price":popularitems_purch_avg,
    "Total Purchase Value":popularitems_total_purch}
items_summary_df = pd.DataFrame(items_summary)

# * Sort the purchase count column in descending order
items_summary_sorted = items_summary_df.sort_values(by ="Purchase Count",ascending = False)

# * Optional: give the displayed data cleaner formatting
items_summary_sorted["Average Purchase Price"] = popularitems_purch_avg.apply("${:.2f}".format)
items_summary_sorted["Total Purchase Value"]=popularitems_total_purch.apply("${:.2f}".format)

# * Display a preview of the summary data frame
items_summary_sorted.head()

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

In [11]:
# * Sort the above table by total purchase value in descending order
items_purchase_data_indexed = purchase_data_df.set_index(["Item ID","Item Name"])
popularitems_df = items_purchase_data_indexed.groupby(["Item ID","Item Name"])

# Perform calculations to obtain purchase count, average item price, and total purchase value
popularitems_purch_count = popularitems_df["Purchase ID"].count()
popularitems_purch_avg = popularitems_df["Price"].mean()
popularitems_total_purch = popularitems_df["Price"].sum()

# * Create a summary data frame to hold the results
items_summary = {"Purchase Count":popularitems_purch_count,
    "Average Purchase Price":popularitems_purch_avg,
    "Total Purchase Value":popularitems_total_purch}
items_summary_df = pd.DataFrame(items_summary)

# * Sort the total purchase value in descending order
items_summary_sorted = items_summary_df.sort_values(by ="Total Purchase Value",ascending = False)

# * Optional: give the displayed data cleaner formatting
items_summary_sorted["Average Purchase Price"] = popularitems_purch_avg.apply("${:.2f}".format)
items_summary_sorted["Total Purchase Value"]=popularitems_total_purch.apply("${:.2f}".format)

# * Display a preview of the data frame
items_summary_sorted.head()

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
